In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import itertools

In [2]:
simplify_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'
gtfs_dir = '/Volumes/Data2/RST/notebook/GTFS/TTC_2016-10-03/'

In [3]:
simplify_dir = 'D:/RST/RST/notebook/GTFS/TTC_2016-10-03/GIS Simplify/'

gtfs_dir = 'D:/RST/notebook/GTFS/TTC_2016-10-03/'

In [4]:
ward_int = pd.read_csv('D:/RST/notebook/GIS/int_wards.csv')
ward_int['INT_ID_STR'] = ward_int['INT_ID'].astype(str)

In [5]:
period = 'MD'
period_hr = 11

In [6]:
G = nx.read_gexf('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.gexf')
graph_data = pd.read_csv('D:/RST/notebook/networks/' + period +'-RM-16-singleDi.csv')
stop_times_od = pd.read_csv(gtfs_dir + 'stop_times_full.csv')

In [7]:
route_dir = stop_times_od[stop_times_od['hr_o'] == period_hr]
route_dir = route_dir[['INT_ID_o', 'INT_ID_d', 'route_short_name']].drop_duplicates()
route_dir = route_dir.groupby(['INT_ID_o', 'INT_ID_d'])['route_short_name'].apply(list).reset_index()
route_dir = route_dir.merge(graph_data)

In [8]:
def func(row):

    single_cycle = row.copy()
    single_cycle.append(single_cycle[0])
    single_cycle.append(None)


    cycle_pairs = list(zip(single_cycle [::1], single_cycle [1::1]))[:-1]
    cycle_pairs = [list(map(int, sublist)) for sublist in cycle_pairs]
    cycle_df = route_dir.merge(pd.DataFrame.from_records(cycle_pairs, columns = ['INT_ID_o', 'INT_ID_d']))

    first_route = cycle_df.iloc[0,2]

    #check for cycles where it can be traversed by 1 route, ie eliminate out and back trips
    for route in first_route:
        if len(cycle_df[cycle_df['route_short_name'].apply(lambda x: route in x)]) == len(cycle_df):
            return ward,0,0,1
            break
        else:
            return ward, 1, cycle_df['scaled_freq'].min(), 0

In [9]:
result_list = []
ward_n = []

for ward in range(1,26,1):

    G_ward = G.copy()


    G_ward.remove_nodes_from(list(ward_int[ward_int['AREA_SHORT_CODE'] != ward]['INT_ID_STR']))

    N = G_ward.size()

    #find cycles
    cycle_list = list(nx.simple_cycles(G_ward))
    print('Ward ' + str(ward) + ', Size ' + str(N) + ' Found '+ str(len(cycle_list)) + ' cycles')

    temp_list = list(map(func, cycle_list))

    result_list.append(temp_list)
    ward_n.append([ward, N])


Ward 1, Size 378 Found 108805 cycles
Ward 2, Size 321 Found 226 cycles
Ward 3, Size 406 Found 433988 cycles
Ward 4, Size 210 Found 2037 cycles
Ward 5, Size 359 Found 63341 cycles
Ward 6, Size 293 Found 1396 cycles
Ward 7, Size 229 Found 230 cycles
Ward 8, Size 267 Found 509 cycles
Ward 9, Size 187 Found 351 cycles
Ward 10, Size 120 Found 98 cycles
Ward 11, Size 178 Found 1919 cycles
Ward 12, Size 144 Found 70 cycles
Ward 13, Size 124 Found 118 cycles
Ward 14, Size 266 Found 7815 cycles
Ward 15, Size 248 Found 130 cycles
Ward 16, Size 190 Found 225 cycles
Ward 17, Size 204 Found 313 cycles
Ward 18, Size 206 Found 231 cycles
Ward 19, Size 237 Found 161 cycles
Ward 20, Size 321 Found 2143 cycles
Ward 21, Size 267 Found 594 cycles
Ward 22, Size 220 Found 9414 cycles
Ward 23, Size 265 Found 23392 cycles
Ward 24, Size 144 Found 71 cycles
Ward 25, Size 270 Found 134 cycles


In [10]:
combined = list(itertools.chain.from_iterable(result_list))
result_df = pd.DataFrame.from_records(
    combined, columns = ['ward', 'cycles', 'scaled_cycles', 'removed_cycles']).groupby(
    'ward').sum().reset_index()
result_df = result_df.merge(pd.DataFrame.from_records(ward_n, columns = ['ward', 'N']))
result_df

,ward,cycles,scaled_cycles,removed_cycles,N
0,1,108632,4083.6250,173,378
1,2,72,2.6625,154,321
2,3,433806,10982.1750,182,406
3,4,1942,50.1375,95,210
4,5,63187,1342.6875,154,359
5,6,1272,38.6625,124,293
6,7,142,7.3625,88,229
7,8,381,10.1625,128,267
8,9,263,8.6375,88,187
9,10,47,3.8500,51,120


In [11]:
result_df['cycle_availability'] = result_df['scaled_cycles']/(2 * result_df['N'] + 5)

In [12]:
result_df = result_df.merge(ward_int[['AREA_NAME', 'AREA_SHORT_CODE']].drop_duplicates(), left_on = ['ward'], 
                 right_on = ['AREA_SHORT_CODE']).rename(columns = {'AREA_NAME':'ward_name'}).drop(
    columns = ['AREA_SHORT_CODE'])
result_df.sort_values(by = 'cycle_availability', ascending = False)

,ward,cycles,scaled_cycles,removed_cycles,N,cycle_availability,ward_name
2,3,433806,10982.1750,182,406,13.442075,Etobicoke-Lakeshore
0,1,108632,4083.6250,173,378,5.366130,Etobicoke North
4,5,63187,1342.6875,154,359,1.857106,York South-Weston
22,23,23275,616.6125,117,265,1.152547,Scarborough North
21,22,9312,241.3000,102,220,0.542247,Scarborough-Agincourt
13,14,7690,249.8250,125,266,0.465223,Toronto-Danforth
10,11,1840,49.8000,79,178,0.137950,University-Rosedale
3,4,1942,50.1375,95,210,0.117971,Parkdale-High Park
19,20,1998,54.7000,145,321,0.084544,Scarborough Southwest
5,6,1272,38.6625,124,293,0.065419,York Centre


In [13]:
result_df.to_csv('cycles-' + period + '.csv', index = False)